In [2]:
import requests as rq
import json
import pandas as pd
from tqdm import tqdm
import time
import pickle
import numpy as np
from datetime import datetime

## Get Data

### Get Room List From Subway

In [3]:
def get(url):
    res = rq.get(url)
    if res.status_code == 200:
        return json.loads(res.text)
    else:
        print(url, res.status_code)
        return None

In [4]:
def get_room_ids(subway_info):
    subway_id = subway_info['id']
    subway_name = subway_info['name']
    room_ids = []
    visited = []
    room_list_json = get(f"https://apis.zigbang.com/v3/items/ad/{subway_id}")
    for i in room_list_json['list_items']:
        if i.get('simple_item') and i['simple_item']['item_id'] not in visited:
            data = {}
            data['item_id'] = i['simple_item']['item_id']
            data['section_type'] = i['section_type']
            data['name'] = subway_name
            room_ids.append(data)
            visited.append(data['item_id'])
    return room_ids

In [5]:
today = datetime.now().strftime('%y%m%d')

In [12]:
url_subway_all = 'https://apis.zigbang.com/property/biglab/subway/all'
subway_ids = get(url_subway_all)
# subway_seoul_ids = [x for x in subway_ids if x['local1'] == '서울특별시']
wish_ids = [x for x in subway_ids if x['name'] == '오이도역' or x['name'] == '정왕역' or x['name'] == '신길온천역' or x['name'] == '안산역' or x['name'] == '달월역' or x['name'] == '월곶역' or x['name'] == '소래포구']
# len(subway_seoul_ids)
len(wish_ids)

5

In [13]:
room_ids = []
for ssi in wish_ids:
    # print(ssi['name'])
    print(ssi) # test
    data = get_room_ids(ssi)
    room_ids.extend(data)
    time.sleep(2)

{'id': 243, 'name': '안산역', 'description': '4호선,수인분당선', 'subway_area': '수도권', 'local1': '경기도', 'lat': 37.3270301818848, 'lng': 126.789558410645, 'lines': [{'short_name': '4', 'color': '#00A2D1'}, {'short_name': '수인분당', 'color': '#F7D056'}]}
{'id': 245, 'name': '정왕역', 'description': '4호선,수인분당선', 'subway_area': '수도권', 'local1': '경기도', 'lat': 37.351816, 'lng': 126.742844, 'lines': [{'short_name': '4', 'color': '#00A2D1'}, {'short_name': '수인분당', 'color': '#F7D056'}]}
{'id': 246, 'name': '오이도역', 'description': '4호선,수인분당선', 'subway_area': '수도권', 'local1': '경기도', 'lat': 37.3616256713867, 'lng': 126.738433837891, 'lines': [{'short_name': '4', 'color': '#00A2D1'}, {'short_name': '수인분당', 'color': '#F7D056'}]}
{'id': 291, 'name': '월곶역', 'description': '수인분당선', 'subway_area': '수도권', 'local1': '경기도', 'lat': 37.3913612365723, 'lng': 126.742538452148, 'lines': [{'short_name': '수인분당', 'color': '#F7D056'}]}
{'id': 1074, 'name': '달월역', 'description': '수인분당선', 'subway_area': '수도권', 'local1': '경기도', 'lat':

In [15]:
room_id_df = pd.DataFrame(room_ids)
room_id_df.to_csv(f'data_subway/room_ids_{today}.csv', index=False)

In [16]:
subway_info = []
subway_line_info = []
for ssi in wish_ids:
    ssi_cop = ssi.copy()
    lines = ssi_cop.pop('lines')
    subway_info.append(ssi_cop)
    for line in lines:
        data = {}
        data['name'] = ssi['name']
        data['line'] = line['short_name']
        subway_line_info.append(data)

subway_info = pd.DataFrame(subway_info)
subway_line_info = pd.DataFrame(subway_line_info)

In [17]:
subway_info.to_csv(f'data_subway/subway_info_{today}.csv', index=False)
subway_line_info.to_csv(f'data_subway/subway_line_info_{today}.csv', index=False)

### Get Room info By Subway

In [18]:
class Subway:
    def __init__(self, subway_line_info):
        self.subway_line_info = subway_line_info
    
    def get_subway_from_line(self, line):
        if isinstance(line, int):
            line = str(line)
        subways = self.subway_line_info[self.subway_line_info['line'] == line]['name'].values
        return subways
    

In [19]:
len(subway_line_info['line'].unique())

2

In [20]:
sw = Subway(subway_line_info)
line_1 = sw.get_subway_from_line(1)
room_1 = room_id_df[room_id_df['name'].isin(line_1)]

### Get Room Info All

In [21]:
room_id_df = pd.read_csv(f'data_subway/room_ids_{today}.csv')
subway_info = pd.read_csv(f'data_subway/subway_info_{today}.csv')
subway_line_info = pd.read_csv(f'data_subway/subway_line_info_{today}.csv')

In [27]:
ids = room_id_df['item_id'].unique()
# len(ids)

635

In [23]:
start = 736  # 현재 저장된 데이터 개수
ids_ = ids[start:]

In [25]:
room_info_list = []
for idx, room_id in tqdm(enumerate(ids)):
    room_info_url = f"https://apis.zigbang.com/v2/items/{room_id}"
    room_info = get(room_info_url)
    room_info_list.append(room_info)
    time.sleep(np.random.random()*0.5 + 0.2)
    if idx % 500 == 499:
        with open(f'data_room/room_info_{start+idx+1}_{today}.pkl', 'wb') as f:
            pickle.dump(room_info_list[idx-499:idx+1], f)

635it [06:48,  1.55it/s]


In [26]:
with open(f'data_room/room_info_{start+idx+1}_{today}.pkl', 'wb') as f:
    pickle.dump(room_info_list[idx+1-(idx+1) % 500: idx+1], f)

## Merge

In [28]:
from glob import glob
import re

In [30]:
target_date = '230911'
data_filenames = glob('data_room/*')
data_filenames = [x for x in data_filenames if target_date in x]
data = []
for fn in tqdm(data_filenames):
    with open(fn, 'rb') as f:
        data.extend(pickle.load(f)) 

100%|██████████| 2/2 [00:00<00:00, 52.64it/s]


In [86]:
def preprocess(df_raw):
    room_info = df_raw[COLS]
    # room_info = room_info.replace('서울특별시', '서울시')
    # room_info = room_info[room_info['local1'] == '서울시'].reset_index(drop=True)
    return room_info

def unify_date_format(date):
    date_converted = re.sub('[^0-9]', '', date)
    return date_converted

def get_year_from_date(date):
    date = re.sub('[^0-9]', '', date)
    if len(date) < 4:
        return -1
    head = int(date[:4])
    if head > 2022:  # 1980.01.01을 80.01.01 이렇게 표현한 경우
        return int('19' + date[:2])
    elif head < 1900:  # 2002.01.01을 02.01.01 이렇게 표현한 경우
        return int('20' + date[:2])
    else:
        return int(date[:4])

In [87]:
COLS = ['item_id', 'user_no','sales_type','보증금액','월세금액','전용면적_m2','공급면적_m2','local1', 'local2', 'local3',
        'random_location', 'floor', 'updated_at','approve_date', 'residence_type']

df_raw = pd.DataFrame([x['item'] for x in data])

room_info = preprocess(df_raw)
# print(room_info)

      item_id   user_no sales_type  보증금액  월세금액  전용면적_m2  공급면적_m2 local1  \
0    37742494  14496807         월세   300  36.0    23.14    23.14    경기도   
1    37894792  17884077         월세   300  30.0    16.53    16.53    경기도   
2    37886936  15423928         전세  8500   NaN    39.67    39.67    경기도   
3    37978949  14496807         월세   300  36.0    23.14    23.14    경기도   
4    37802342  14496807         전세  8500   NaN    36.36    36.36    경기도   
..        ...       ...        ...   ...   ...      ...      ...    ...   
630  37906780   1328580         월세   500  50.0    29.88    47.75    경기도   
631  37978707   3868336         월세  1000  65.0    37.36    81.62    경기도   
632  37906774   1328580         월세   300  55.0    29.88    51.03    경기도   
633  38003799   3868336         월세   500  50.0    25.79    56.88    경기도   
634  38028091   3868336         월세   300  33.0    16.52    23.14    경기도   

      local2 local3                        random_location floor  \
0    안산시 단원구    신길동   37.332708

In [90]:
room_info['approve_year'] = room_info.apply(lambda x:get_year_from_date(x['approve_date']), axis=1)
room_info.loc[room_info['approve_year'] == -1, 'approve_year'] = \
    int(room_info[room_info['approve_year'] != -1]['approve_year'].mean())
room_info['random_lat'] = room_info.apply(lambda x:x['random_location'].split(',')[0], axis=1)
room_info['random_long'] = room_info.apply(lambda x:x['random_location'].split(',')[1], axis=1)
room_info['is_underground'] = 0
room_info.loc[room_info['floor'] == '반지하', 'is_underground'] = 1
room_info.to_csv(f'data_preprocessed/room_info_{target_date}.csv', index=False)

C:\Users\chosh\AppData\Local\Temp\ipykernel_111668\2196643461.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_info['approve_year'] = room_info.apply(lambda x:get_year_from_date(x['approve_date']), axis=1)
C:\Users\chosh\AppData\Local\Temp\ipykernel_111668\2196643461.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_info['random_lat'] = room_info.apply(lambda x:x['random_location'].split(',')[0], axis=1)
C:\Users\chosh\AppData\Local\Temp\ipykernel_111668\2196643461.py:5: SettingWithCopyWarnin

In [92]:
room_info.to_excel('data_preprocessed/room_info_230911.xlsx', index=False)

## Trash

In [ ]:
req1 = rq.get("https://apis.zigbang.com/v3/items/ad/17?deposit_s=0&detail=false&domain=zigbang&floor=1~%7Crooftop%7Csemibase&radius=1&rent_s=0&sales_type=")
req2 = rq.get("https://apis.zigbang.com/v3/items/ad/17")

In [ ]:
import os
path = 'data_room/'
date = '221201'
files = os.listdir(path)
for fn in files:
    if date in fn:
        num = int(fn.split('_')[-2]) - start
        new_fn = f'room_info_{num}_{date}.pkl'    
        os.rename(path+fn, path+new_fn)